<a href="https://colab.research.google.com/github/Gregrs400/cmpsc472Project2/blob/threads/cmpsc472Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
%%writefile cmpsc472Project2.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/wait.h>
#include <sys/time.h>
#include <sys/resource.h>
#include <time.h>
#include <stdbool.h>
#include <semaphore.h>
#include <pthread.h>

#define MAX_COLUMNS 49

/*
TODO:
- add global pool of resources to fight fires. This should be an int with an arbritrary value, e.g. 30 resources
- add fire fighting resources throughout the landscape
- determine whether to have multiple processes or multiple threads deal with the wildfires - Note: have parent process act as dispatch, have a child process search for wildfires, have
multiple threads from that child process fight the wildfires
- implement inter-process communication
- use semaphores/mutexes to lock critical sections
- have logic for extinguishing wildfires, add resources back to global resource pool when a fire is extinguished
*/

int global_resources = 50;
sem_t semaphore;

struct Landscape {
    int rows;
    int columns;
    int (*grid)[MAX_COLUMNS];
};

void printLandscape(struct Landscape *landscape);

struct Fire {
  int row;
  int col;
  int priority;
  int requiredResources;
};

struct ThreadArgs {
  int beginRow;
  int endRow;
  // Pipe array for IPC
  int fireToDispatch;
  struct Landscape *landscape;
};


void* searchForFires(void* args) {
  struct ThreadArgs* threadArgs = (struct ThreadArgs*)args;
  int beginRow = threadArgs->beginRow;
  int endRow = threadArgs->endRow;
  int fireToDispatch = threadArgs->fireToDispatch;
  struct Landscape *landscape;

  // Search landscape for fires
  for (int i = beginRow; i < endRow; i++) { // Start the search at beginRow, go up to endRow
    for (int j = 0; j < landscape ->columns; j++) { // Go through all columns
      if (landscape -> grid[i][j] == 1) {
        // Lock semaphore before writing to parent process
        sem_wait(&semaphore);
        // Communicate these coordinates to parent process
        write(fireToDispatch, &i, sizeof(i));
        write(fireToDispatch, &j, sizeof(j));
        // Unlock semaphore after writing to parent process
        sem_post(&semaphore);
      }
    }
  }

}

// Function to set a random position in the array to 1, which represents a wildfire.
struct Fire setFire(struct Landscape *landscape, int min, int max) {
  struct Fire fire;
  // Seed the random number generator
  srand(time(NULL));
  // Randomly select a row
  fire.row = rand() % (max - min + 1) + min;
  // Randomly select a column
  fire.col = rand() % (max - min + 1) + min;
  // Set the value at the selected row and column to 1
  landscape -> grid[fire.row][fire.col] = 1;
  printf("array value at position %d,%d: %d\n", fire.row, fire.col, landscape -> grid[fire.row][fire.col]);
  // Randomly assign a priority. Lower priorities have highest priority.
  fire.priority = rand() % (max - min + 1) + min;
  // Randomly assign a required number of resources to extinguish the fire.
  fire.requiredResources = rand() % (max - min + 1) + min;
  printf("Fire has priority %d and requires %d resources.\n", fire.priority, fire.requiredResources);
  return fire;
}

// Prints out the entire landscape array.
void printLandscape(struct Landscape *landscape) {
  for (int i = 0; i < landscape->rows; i++) {
    for (int j = 0; j < landscape->columns; j++) {
      if (landscape -> grid[i][j] == 1)
      {
        printf("\033[1;31m%d\033[0m ", landscape->grid[i][j]);
        continue;
      }
      else if (landscape -> grid[i][j] == 0)
      {
        printf("\033[1;32m%d\033[0m ", landscape->grid[i][j]);
        continue;
      }
      else if (landscape -> grid[i][j] == 5)
      {
        printf("\033[1;34m%d\033[0m ", landscape->grid[i][j]);
        continue;
      }
      printf("%d ", landscape->grid[i][j]);
    }
    // Print a newline at the end of each row.
    printf("\n");
  }
}


int main() {
  // Create pipes for IPC
  int pipe_fd[2];
  if (pipe(pipe_fd) == -1) {
    perror("pipe");
    exit(EXIT_FAILURE);
  }

  struct Landscape *landscape = (struct Landscape *)malloc(sizeof(struct Landscape));

  if (landscape == NULL) {
      fprintf(stderr, "Memory allocation failed\n");
      return 1;
  }

  // Define number of rows
  landscape -> rows = 49;
  // Define number of columns
  landscape -> columns = 49;
  // Create array to represent the landscape
  int landscapeGrid[landscape -> rows][landscape -> columns];
  // setting the array to the landscape struct
  landscape -> grid = landscapeGrid;
  int totalGridRows = landscape -> rows;
  int totalGridColumns = landscape -> columns;
  // Fill array with 0s
  for (int i = 0; i < totalGridRows; i++)
  {
      for (int j = 0; j < totalGridColumns; j++)
      {
        if (i == totalGridRows / 2)
        {
            landscape -> grid[i][j] = 5;
            if (j == totalGridColumns / 2)
            {
              landscape -> grid[i][j] = 5;
              continue;
            }
        }
        landscape -> grid[i][j] = 0;
      }
  }

  // Number of fires to be set.
  int numOfFires = 5;
  // Amount of time to wait between setting fires.
  int waitTime;
  // Create array of Fire structs to store info on each fire set.
  struct Fire fires[numOfFires];
  // For loop to set the fires.
  for (int j = 0; j < numOfFires; j++) {
    // Stores attributes (row, col, priority, number of required resources) about the fire in jth element of Fire struct array.
    fires[j] = setFire(landscape, 0, 19);
    printLandscape(landscape);
    printf("\n");

    // Seed random number generator
    srand(time(NULL));
    // Wait between 1 and 10 seconds before setting another fire.
    waitTime = rand() % (10 - 1 + 1) + 1;
    // Wait a random amount of time before setting another fire.
    sleep(waitTime);
  }

  // Now that all fires are set, create a child process to search for fires
  pid_t child_pid = fork();
  if (child_pid == 0) {
    // Counter to iterate through fires array
    int k = 0;
    // Search for fires in the landscape.
    // If one is found, use IPC to communicate that with the parent process.
    // Close read end of pipe
    close(pipe_fd[0]);

    // Initialize semaphore
    sem_init(&semaphore, 0, 1);
    // Thread IDs
    pthread_t threads[2];
    struct ThreadArgs threadArgs[2];

    // Set the args for first thread.
    threadArgs[0].beginRow = 0;
    threadArgs[0].endRow = totalGridRows / 2;
    threadArgs[0].landscape = landscape;
    //threadArgs[0].fireToDispatch = pipe_fd[0];
    //Note: I think this will use the same pipe for both threads.
    // Might have to init pipes in the thread function? Although this may not communicate with the parent process.
    threadArgs[0].fireToDispatch = pipe_fd[1];

    // Set the args for the second thread.
    threadArgs[1].beginRow = totalGridRows / 2;
    threadArgs[1].endRow = totalGridRows;
    threadArgs[1].landscape = landscape;
    //threadArgs[1].fireToDispatch[1] = pipe_fd[0];
    threadArgs[1].fireToDispatch = pipe_fd[1];

    // Have threads search for fires in the landscape
    pthread_create(&threads[0], NULL, searchForFires, &threadArgs[0]);
    pthread_create(&threads[1], NULL, searchForFires, &threadArgs[1]);

    // Wait for threads to finish
    pthread_join(threads[0], NULL);
    pthread_join(threads[1], NULL);
    // After child finishes search of landscape, notify the parent
    int complete = -1;
    write(pipe_fd[1], &complete, sizeof(-1));
    write(pipe_fd[1], &complete, sizeof(-1));

    // Close write end of pipe
    close(pipe_fd[1]);

  } else if (child_pid > 0) {
    // Parent process
    // Variables to store the row, column, requiredResources passed from child process
    int r, c, rr;
    // Close write end of pipe
    close(pipe_fd[1]);

    // While child process searches for fires, listen for IPC from child process
    while (true) {
      read(pipe_fd[0], &r, sizeof(r));
      if (r != -1)
      {
        printf("Fire row: %d\n", r);
      }
      read(pipe_fd[0], &c, sizeof(c));
      if (c != -1)
      {
        printf("Fire col: %d\n", c);
        read(pipe_fd[0], &rr, sizeof(rr));
        printf("Num of resources: %d\n", rr);
      }

      printf("\n");

      if (r == -1 && c == -1) {
        // Child process has finished searching for fires
        break;
      }
    }
    // Wait to ensure child process has finished executing.
    wait(NULL);

  } else {
    perror("Failed to create child process.");
    exit(EXIT_FAILURE);
  }
  // Destroy semaphore
  sem_destroy(&semaphore);

  // Free the memory allocated to the landscape struct
  free(landscape);
  return 0;
}

Overwriting cmpsc472Project2.c


In [36]:
%%shell
gcc cmpsc472Project2.c -o cmpsc472Project2
./cmpsc472Project2

Streaming output truncated to the last 5000 lines.

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fire row: 0
Fire col: 1
Num of resources: 0

Fir

CalledProcessError: Command 'gcc cmpsc472Project2.c -o cmpsc472Project2
./cmpsc472Project2
' died with <Signals.SIGINT: 2>.